In [1]:
import gym
import ray

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [3]:
# Initialize Ray
ray.shutdown()

In [4]:
from rayflow import Rayflow
r = Rayflow.load("test_input.yml", minio_volume_path="./minio_volume/", mlflow_volume_path="./mlflow_volume/",no_ray=True)

HOST INFO {'192.168.1.65': {'ansible_python_interpreter': '~/anaconda3/envs/py37/bin/python', 'ansible_connection': 'local'}}


/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (127.0.0.1) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


In [5]:
r.start()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (192.168.1.65) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-ke

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-73391j6kvolc9/tmpjtvmcaoy TO /home/shyam/.ansible/tmp/ansible-tmp-1601338611.1414971-184957078717966/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601338611.1414971-184957078717966/ /home/shyam/.ansible/tmp/ansible-tmp-1601338611.1414971-184957078717966/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601338611.1414971-184957078717966/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601338611.1414971-184957078717966/ > /dev/null 2>&1 && sleep 0'
ok: [192.168.1.65] => {
    "changed": false,
    "invocation": {
        "module_args": {
            "api_version": "auto",
    

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-73585f512who2/tmpzy11otnb TO /home/shyam/.ansible/tmp/ansible-tmp-1601338615.3467462-115669116923357/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601338615.3467462-115669116923357/ /home/shyam/.ansible/tmp/ansible-tmp-1601338615.3467462-115669116923357/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601338615.3467462-115669116923357/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601338615.3467462-115669116923357/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",

2020-09-28 17:16:56,997	INFO resource_spec.py:231 -- Starting Ray with 32.47 GiB memory available for workers and up to 16.25 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601338616.1731608-101621329501515/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
            "Args": [
                "server",
                "--backend-store-uri",
                "/mlflow",
                "--host",
                "192.168.1.65",
                "-p",
                "5000",
                "--default-artifact-root",
                "s3://artifacts/"
            ],
            "Config": {
                "AttachStderr": false,
                "AttachStdin": false,
                "AttachStdout": false,
                "Cmd": [
                    "mlflow",
                    "server",
                    "--backend-store-uri",
                    "/mlflow",
                    "--host",
                    "192.168.1.65",
                    "-p",
                    "5000

2020-09-28 17:16:57,380	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


In [6]:
from minerl_rllib.envs import register
register()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [16]:
from ray.rllib.agents.ppo import PPOTrainer

train_name = "mlflow_train"
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"MINERL_WORK_PLEASE",
    "create_new_experiment": True
}
trainer_config = {
        "env": "MineRLNavigateDenseVectorObf-v0",
         "num_gpus": 1,
         "num_workers": 1,
         "num_envs_per_worker": 1,
         "eager": False,
         "use_pytorch": True,
         'monitor':True, 
}
saver_config = {
        "checkpoint_on_end": True,
        "checkpoint_freq": 2
}
stop = {
        "training_iteration": 5,
        "timesteps_total": 100000,
        "episode_reward_mean": 200.0,
}
tune_args = {
        "checkpoint_at_end": True,
        "checkpoint_freq": 2
}

config = {
    "trainer_config":trainer_config,
    "logging_config": mlflow_logging_config
}

In [17]:
from rayflow.tune import Tune
t = Tune()

In [18]:
import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer

In [19]:
run_id, output_config, experiment_analysis = t.run_training(
               trainer=PPOTrainer, 
               name=train_name, 
               stop=stop, 
               mlflow_logging_config=mlflow_logging_config, 
               trainer_config=trainer_config,
               saver_config=saver_config,
               tune_args=tune_args,
               )

2020-09-28 17:17:30,764	WARNING tune.py:325 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc
MLFlowTrainer_29f17_00000,RUNNING,


(pid=74017) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/utils/util.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
(pid=74017)   from collections import defaultdict, deque, Mapping, Sequence
(pid=74017) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/tensorflow/python/autograph/utils/testing.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=74017)   import imp
(pid=74017) WARNING:tensorflow:From /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=74017) Instructions for updating:
(pid=74017) non-resource variables are not supported in the long term
(pid=74017) /home/shyam/a

Result for MLFlowTrainer_29f17_00000:
  custom_metrics: {}
  date: 2020-09-28_17-22-30
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: b571bc085df64e58a3607cf726ce0e80
  experiment_tag: '0'
  hostname: kokkgoblin
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 90.62773132324219
        entropy_coeff: 0.0
        kl: 0.021785135846585035
        policy_loss: -0.030136926943669096
        total_loss: 0.09268462799082045
        vf_explained_var: -3.5390257835388184e-07
        vf_loss: 0.11846452997997403
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.1.65
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 18.661818181818184
    gpu_util_percent0: 0.18657142857142855
    ram_ut

Trial name,status,loc,iter,total time (s),ts,reward
MLFlowTrainer_29f17_00000,RUNNING,192.168.1.65:74017,1,295.432,4000,nan


2020-09-28 17:22:32,265	ERROR trial_runner.py:523 -- Trial MLFlowTrainer_29f17_00000: Error processing event.
Traceback (most recent call last):
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 471, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 430, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/worker.py", line 1538, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TypeError): ray::MLFlowTrainer.train() (pid=74017, ip=192.168.1.65)
  File "python/ray/_raylet.pyx", line 479, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 432, in ray._raylet.execute_task.function_executor
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/tune/trainabl

TuneError: ('Trials did not complete', [MLFlowTrainer_29f17_00000])

(pid=74012) Attempted to send kill command to minecraft process and failed.
(pid=74012) *** Aborted at 1601338953 (unix time) try "date -d @1601338953" if you are using GNU date ***
(pid=74012) PC: @                0x0 (unknown)
(pid=74012) *** SIGSEGV (@0x7f9582ffd9d0) received by PID 74692 (TID 0x7f95a6570740) from PID 18446744071612389840; stack trace: ***
(pid=74012)     @     0x7f95a61618a0 (unknown)
(pid=74012)     @     0x7f95a6157bd8 __GI___pthread_timedjoin_ex
(pid=74012)     @     0x7f95a3deb2d3 std::thread::join()
(pid=74012)     @     0x7f95a431c493 ray::gcs::GlobalStateAccessor::Disconnect()
(pid=74012)     @     0x7f95a41bbfbc __pyx_pw_3ray_7_raylet_19GlobalStateAccessor_5disconnect()
(pid=74012)     @     0x55dadb71dea1 _PyMethodDef_RawFastCallKeywords
(pid=74012)     @     0x55dadb71e14f _PyMethodDescr_FastCallKeywords
(pid=74012)     @     0x55dadb779fa9 _PyEval_EvalFrameDefault
(pid=74012)     @     0x55dadb71d07b _PyFunction_FastCallKeywords
(pid=74012)     @     0x5

In [ ]:
r.stop()

### Register MineRL Gym Environment to RLlib

In [ ]:
from minerl_rllib.envs import register

In [ ]:
register()

In [ ]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples
# config = ppo.DEFAULT_CONFIG.copy()
# config["num_gpus"] = 1
# config["num_workers"] = 1
# config["num_envs_per_worker"] = 1
# config["eager"] = False
# trainer = ppo.PPOTrainer(config=config, env="MineRLNavigateDenseVectorObf-v0")


tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

In [ ]:
r.stop()